**File explorer** to find the file you want to analyse with **Statement Processor** to generate all data and visualisations

In [ ]:
%run ProcessxAPISGStatement.ipynb # notebook to process an xAPI-SG statement
%run widgets/selectFileWidget.ipynb
if local:
    global fileBrowser
    #*** !TO BE USED WHEN THE NOTEBOOK IS RUNNED LOCALLY ***
    fileBrowser = FileBrowser()

    def on_file_load(change):
        global location_file, checkboxesPlayersSelected
        players_info.clear()
        # file with xAPI-SG statements
        location_file=fileBrowser.path
        try:
            start_time = datetime.now()
            ## FILE MUST CONTAIN LIST OF XAPI-SG PROFILE TRACES
            ## (traces separated by commas and enclosed by [])
            with open(location_file, 'r', encoding='UTF-8') as f:
                statements_dict = json.load(f, encoding='UTF-8')
                for s in statements_dict:
                    process_xapisg_statement(s, players_info, timeformats)
                end_time = datetime.now()
        except Exception as e:
            with fileBrowser.output:
                print("FILE MUST CONTAIN LIST OF XAPI-SG PROFILE TRACES (traces separated by commas and enclosed by [])")
                print("This file cannot be analyse by the processor per statement xAPI. Please select an another file.")
                clear_output(wait=True)
        with fileBrowser.output:
            print("Process per statement OK")
            print("Displaying all vis...")
            clear_output(wait=True)
            display_checkboxes()
            clear_output(wait=True)
        displayvis(None)
    fileBrowser.buttonRun.on_click(on_file_load)

else:
    global upload_button
    #***  !TO BE USED WHEN THE NOTEBOOK IS HOSTED ON THE WEB ***
    upload_button=widgets.FileUpload(
        description='Upload xAPI data',
        accept='.json',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False,  # True to accept multiple files upload else False
        layout=Layout(width='25%')
    )
    global outCheckboxes, outTabs
    outCheckboxes=widgets.Output()
    outTabs=widgets.Output()
    outF=VBox([outCheckboxes, outTabs])
    app=VBox([upload_button, outF])

    #Observe the file load widget (online version)
    def on_file_upload(change):
        global location_file, checkboxesPlayersSelected
        location_file = (upload_button.metadata[0]['name']) #get the file name from the embedded metadata dict
        f = upload_button.value[location_file]['content'].decode() #extract data byte string and convert to str
        try:
            players_info.clear()
            start_time = datetime.now()
            ## FILE MUST CONTAIN LIST OF XAPI-SG PROFILE TRACES
            ## (traces separated by commas and enclosed by [])
            statements_dict = json.loads(f, encoding='UTF-8')
            for s in statements_dict:
                process_xapisg_statement(s, players_info, timeformats)
            end_time = datetime.now()
        except Exception as e:
            with outCheckboxes:
                print("FILE MUST CONTAIN LIST OF XAPI-SG PROFILE TRACES (traces separated by commas and enclosed by [])")
                print("This file cannot be analyse by the processor per statement xAPI. Please select an another file.")
                clear_output(wait=True)
        with outCheckboxes:
                print("Process per statement OK")
                print("Displaying all vis...")
                clear_output(wait=True)
                display_checkboxes()
                clear_output(wait=True)
        displayvis(None)

    upload_button.observe(on_file_upload, names='value')

def displayvis(change):
    if local:
        with fileBrowser.output2:
            displayAllsVisualisations()
            clear_output(wait=True)
    else:
        with outTabs:
            displayAllsVisualisations()
            clear_output(wait=True)
